In [24]:
from os import path
import tensorflow as tf
from tensorflow.keras import layers, models

In [50]:
# Define the file path and column names for the CSV file
csv_file_path = "/Users/jeevakumar/Desktop/FADML/project/CrisisMMD_v2.0/train_image.csv"
image_col_name = "image"
label_col_name = "label"

# Define the batch size and image size
batch_size = 32
image_size = (224, 224)

# Define the number of classes
num_classes = 3

# Define a function to preprocess the images
def preprocess_image(image_path, label):
    # Load the image file
    image = tf.io.read_file(image_path)
    # Decode the image to a tensor
    image = tf.image.decode_jpeg(image, channels=3)
    # Resize the image to the desired size
    image = tf.image.resize(image, image_size)
    # Convert the pixel values to floats in the range [0, 1]
    image = tf.cast(image, tf.float32) / 255.0
    # Convert the label to a one-hot encoded vector
    label = tf.one_hot(label, depth=num_classes)
    return image, label

# Create a TensorFlow dataset from the CSV file
dataset = tf.data.experimental.CsvDataset(csv_file_path, record_defaults=[tf.string, tf.int32], header=True,
                                          field_delim=',', use_quote_delim=True)

# Map the preprocessing function to the dataset
dataset = dataset.map(preprocess_image)

# Shuffle and batch the dataset
dataset = dataset.shuffle(buffer_size=1000)
dataset = dataset.batch(batch_size)

# Print the first batch of elements in the dataset
for image_batch, label_batch in dataset.take(1):
    print("Image batch shape:", image_batch.shape)
    print("Label batch shape:", label_batch.shape)


Image batch shape: (32, 224, 224, 3)
Label batch shape: (32, 3)


In [57]:
base_model = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
base_model.trainable = False
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [58]:
val = tf.data.experimental.CsvDataset("/Users/jeevakumar/Desktop/FADML/project/CrisisMMD_v2.0/test_image.csv", record_defaults=[tf.string, tf.int32], header=True,
                                          field_delim=',', use_quote_delim=True)

# Map the preprocessing function to the dataset
val = val.map(preprocess_image)

# Shuffle and batch the dataset
val = val.shuffle(buffer_size=1000)
val = val.batch(batch_size)



# Print the first batch of elements in the dataset
for image_batch, label_batch in val.take(1):
    print("Image batch shape:", image_batch.shape)
    print("Label batch shape:", label_batch.shape)


Image batch shape: (32, 224, 224, 3)
Label batch shape: (32, 3)


In [59]:
model.fit(
    dataset,
    validation_data=val,
    epochs=10
)

Epoch 1/10


2023-03-21 23:01:23.855457: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


     78/Unknown - 58s 691ms/step - loss: 2.2423 - accuracy: 0.5697

2023-03-21 23:02:20.745423: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


78/78 [==============================] - 72s 879ms/step - loss: 2.2423 - accuracy: 0.5697 - val_loss: 0.8945 - val_accuracy: 0.6276
Epoch 2/10
78/78 [==============================] - 63s 790ms/step - loss: 0.9664 - accuracy: 0.6248 - val_loss: 0.9075 - val_accuracy: 0.6276
Epoch 3/10
78/78 [==============================] - 68s 853ms/step - loss: 0.9454 - accuracy: 0.6272 - val_loss: 0.9024 - val_accuracy: 0.6276
Epoch 4/10
78/78 [==============================] - 77s 963ms/step - loss: 0.9571 - accuracy: 0.6272 - val_loss: 0.9212 - val_accuracy: 0.6276
Epoch 5/10
78/78 [==============================] - 73s 910ms/step - loss: 0.9402 - accuracy: 0.6272 - val_loss: 0.9239 - val_accuracy: 0.6276
Epoch 6/10
78/78 [==============================] - 66s 827ms/step - loss: 0.9263 - accuracy: 0.6272 - val_loss: 0.8898 - val_accuracy: 0.6276
Epoch 7/10
78/78 [==============================] - 83s 1s/step - loss: 0.9203 - accuracy: 0.6272 - val_loss: 0.9144 - val_accuracy: 0.6276
Epoch 8/10
78